# Zebrafish Dataset - LSD Training

This notebook demonstrates training the Latent State Dynamics (LSD) model on the Zebrafish dataset with perturbation analysis.

**Dataset Information:**
- ~2,068 cells
- ~2,000 highly variable genes
- Simple bifurcation pathway
- Focus on perturbation analysis

## Setup

In [ ]:
import os
import scanpy as sc
import numpy as np
import torch

from sclsd import LSD, LSDConfig, set_all_seeds, clear_pyro_state

SEED = 42
clear_pyro_state()
set_all_seeds(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Load Data

In [ ]:
data_path = "../../data/Zebrafish/perturb_train_adata.h5ad"
adata = sc.read(data_path)
print(f"Loaded {adata.n_obs} cells x {adata.n_vars} genes")

In [ ]:
sc.pl.umap(adata, color='clusters', title='Zebrafish - Cell Types')

## Configure LSD Model

Key hyperparameters for Zebrafish:
- `V_coeff = 5e-3`
- `kl_schedule.af = 3`
- `lr = 2e-3`
- `batch_size = 256`
- `path_len = 40`
- `num_epochs = 250`
- Custom potential network: `[16, 16]`

In [ ]:
cfg = LSDConfig()

cfg.model.V_coeff = 5e-3
cfg.model.z_dim = 10
cfg.model.B_dim = 2
cfg.model.layer_dims.potential = [16, 16]  # Custom potential network

cfg.optimizer.kl_schedule.af = 3
cfg.optimizer.adam.lr = 2e-3

cfg.walks.batch_size = 256
cfg.walks.path_len = 40
cfg.walks.num_walks = 4096
cfg.walks.random_state = SEED

## Initialize and Train

In [ ]:
lsd = LSD(adata, cfg, device=device)
lsd.set_prior_transition(prior_time_key="prior_time")
lsd.prepare_walks()

print(f"Generated {lsd.walks.shape[0]} walks of length {lsd.walks.shape[1]}")

In [ ]:
model_dir = "./zebrafish_model"
os.makedirs(model_dir, exist_ok=True)

lsd.train(
    num_epochs=250,
    save_dir=model_dir,
    save_interval=50,
    random_state=SEED,
)

## Extract and Save Results

In [ ]:
result = lsd.get_adata()
sc.pl.umap(result, color=['lsd_pseudotime', 'potential', 'entropy'], ncols=3)

In [ ]:
lsd.save(dir_path=model_dir, file_name="lsd_model_final.pth")
result.write(os.path.join(model_dir, "result_adata.h5ad"))
print(f"Results saved to {model_dir}")

## Perturbation Analysis

The Zebrafish dataset is particularly useful for studying gene perturbation effects on cell fate.

In [ ]:
# Example perturbation analysis (requires test data with known perturbation genes)
# Uncomment and modify based on your data

# # Load test data with cells to perturb
# test_adata = sc.read("../../data/Zebrafish/perturb_test_adata.h5ad")
# x_test = torch.from_numpy(test_adata.X.toarray()).float()

# # Perform perturbation
# gene_name = "target_gene"  # Replace with actual gene name
# perturbed_fates, unperturbed_fates = lsd.perturb(
#     result,
#     x_test,
#     gene_name=gene_name,
#     cluster_key="clusters",
#     perturbation_level=0,
#     dt_pert=0.2,
#     t_unpert=15,
#     max_perturbations=10,
# )

# # Analyze fate changes
# print(f"Fate changes after {gene_name} knockout:")
# print(f"  Perturbed fates: {np.unique(perturbed_fates, return_counts=True)}")
# print(f"  Unperturbed fates: {np.unique(unperturbed_fates, return_counts=True)}")